In [66]:
import sentencepiece as spm

In [67]:
import pickle
import gzip

with gzip.open('pre_reviews_2.pickle', 'rb') as f:
    pre_reviews = pickle.load(f)

In [97]:
corpus = [x for x in pre_reviews if (len(x[0].split()) < 16 and len(x[0]) < 160)]

In [109]:
corpus[167838]

('garbagemycopbasuraorduresmülltruflaisrifiutitakatakasoporroskatvullismbeturinazaborescombrariessmećeodpadkyskraldafvalruboprügilixofatraszemét',
 0)

In [99]:
input_file = 'spm_input.txt'

with open(input_file, 'w', encoding='utf-8') as f:
    for sent in corpus:
        f.write('{}\n'.format(sent[0]))

In [100]:
input_file = 'spm_input.txt'
templates = '--input={} --model_prefix={} --vocab_size={} --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3'

vocab_size = 5000
prefix = 'reviews'
cmd = templates.format(input_file, prefix, vocab_size)

In [101]:
spm.SentencePieceTrainer.Train(cmd)

True

In [102]:
sp = spm.SentencePieceProcessor()
sp.Load('{}.model'.format(prefix))

True

In [103]:
with open('{}.vocab'.format(prefix), encoding='utf-8') as f:
    vocabs = [doc.strip() for doc in f]

print('num of vocabs = {}'.format(len(vocabs)))

num of vocabs = 5000


In [104]:
sent = corpus[0][0]
extra_options = 'bos:eos' #'reverse:bos:eos'
sp.SetEncodeExtraOptions(extra_options)
sp.EncodeAsPieces(sent)

['<s>',
 '▁we',
 '▁got',
 '▁this',
 '▁gps',
 '▁for',
 '▁my',
 '▁husband',
 '▁who',
 '▁is',
 '▁an',
 '▁o',
 'tr',
 '▁over',
 '▁the',
 '▁road',
 '▁truck',
 'er',
 '</s>']

In [105]:
print("i love you")
print(sp.EncodeAsPieces("i love you"))
print(sp.EncodeAsIds("i love you so much"))
print(sp.EncodeAsIds("i hate you"))
print(vocabs[0])

i love you
['<s>', '▁i', '▁love', '▁you', '</s>']
[1, 5, 174, 21, 34, 98, 2]
[1, 5, 1002, 21, 2]
<pad>	0


In [106]:
encoded_reviews = [[sp.EncodeAsIds(sent[0]), sent[1]] for sent in corpus]

In [107]:
print(encoded_reviews[1])

[[1, 95, 28, 76, 6, 95, 2], 1]


In [108]:
# save and compress.
with gzip.open('encoded_reviews.pickle', 'wb') as f:
    pickle.dump(encoded_reviews, f)

임시

In [86]:
encoded_reviews[1][0]

[1, 103, 34, 61, 7, 103, 2]

In [52]:
output_file = 'real_data.txt'

with open(output_file, 'w', encoding='utf-8') as f:
    count = 0
    for review in encoded_reviews:
        sent = review[0]
        if(len(sent) != 20):
            continue
        else:
            count += 1
            f.write('{}\n'.format(" ".join(list(map(str, sent)))))
            if(count == 10000):
                break

In [46]:
sp.DecodeIds([6, 163, 2760, 2006, 2359, 1709, 3236,3425, 2181])

'it should knows firewire belt select grade thankfully dsl'

In [59]:
f = open("generator_sample0.09.txt", 'r')
lines = f.readlines()
f.close()

In [60]:
sample = [list(map(int, line[:-1].split())) for line in lines]

In [61]:
file = 'real_sample.txt'

with open(file, 'w', encoding='utf-8') as f:
    for sent in sample:
        f.write('{}\n'.format(sp.DecodeIds(sent)))

In [59]:
sp.DecodeIds(list(map(int, '1 4 22 5 1182 198 252 554 493 1501 128 28 4 68 184 5 351 64 67 2'.split(' ')

'i have a ham radio license so i do know a bit about them'